# Multiturn injection analysis — multibunch comparison

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import proplot as pplt
from ipywidgets import interact
from ipywidgets import widgets

In [ ]:
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["grid"] = False
pplt.rc["figure.facecolor"] = "white"

## Load data

In [ ]:
timestamps = {
    "sc_off": 240514171357,
    "sc_on ": 240514172003,
}
keys = list(timestamps)

script_names = {
    keys[0]: "paint_prod",
    keys[1]: "paint_prod",
}

In [ ]:
input_dirs = {}
for key in keys:
    timestamp = timestamps[key]
    script_name = script_names[key]
    input_dir = f"./outputs/{script_name}/{timestamp}/"
    input_dirs[key] = input_dir

In [ ]:
def get_bunch_filenames(input_dir: str) -> list[str]:
    filenames = os.listdir(input_dir)
    filenames = sorted(filenames)
    filenames = [f for f in filenames if f.startswith("bunch")]
    filenames = [os.path.join(input_dir, f) for f in filenames]
    return filenames


def load_bunch(filename: str, subtract_mean: bool = True) -> np.ndarray:
    X = np.loadtxt(filename, comments="%", usecols=range(6))
    X[:, :4] *= 1000.0
    X[:, 5] *= 1000.0
    if subtract_mean:
        X -= np.mean(X, axis=0)    
    turn = int(filename.split("_")[-1].split(".dat")[0])
    return (X, turn)

In [ ]:
bunches = {}
bunch_turns = {}
for key in keys:
    print(key)
    bunches[key] = []
    bunch_turns[key] = []    
    for filename in get_bunch_filenames(input_dirs[key]):
        X, turn = load_bunch(filename)
        bunches[key].append(X)
        bunch_turns[key].append(turn)

        print("turn =", turn)

## Scalars

## Phase space distribution

In [ ]:
dims = ["x", "xp", "y", "yp", "z", "dE"]
units = ["mm", "mrad", "mm", "mrad", "m", "MeV"]
labels = [f"{dim} [{unit}]" for dim, unit in zip(dims, units)]

xmaxs = 4.0 * np.std(bunches[keys[0]][-1], axis=0)
limits = [(-xmax, xmax) for xmax in xmaxs]

### Interactive 2D projections

In [ ]:
@interact(
    key=keys,
    dim1=widgets.Dropdown(options=dims, value=dims[0]),
    dim2=widgets.Dropdown(options=dims, value=dims[1]),
    index=widgets.IntSlider(min=0, max=(len(bunches[keys[0]]) - 1), value=0),
    bins=widgets.IntSlider(min=32, max=128, value=64),
)
def update(key: str, dim1: str, dim2: str, index: int, bins: int):
    if dim1 == dim2:
        return

    axis = [dims.index(dim) for dim in [dim1, dim2]]
    axis = tuple(axis)
    
    X = bunches[key][index]

    fig, ax = pplt.subplots()
    ax.hist2d(
        X[:, axis[0]], 
        X[:, axis[1]], 
        bins=bins,
        range=[limits[axis[0]], limits[axis[1]]],
    )
    ax.format(xlabel=labels[axis[0]], ylabel=labels[axis[1]])
    ax.format(title=f"turn = {bunch_turns[key][index]}")

    paxs = [ax.panel_axes(loc) for loc in ["top", "right"]]
    for pax in paxs:
        pax.format(xspineloc="bottom", yspineloc="left")
    kws = dict(bins=bins, density=False, color="black", histtype="step", lw=1.25)
    paxs[0].hist( X[:, axis[0]], range=limits[axis[0]], **kws)
    paxs[1].histh(X[:, axis[1]], range=limits[axis[1]], **kws)
    plt.show()

### Interactive 2D projections side-by-side

In [ ]:
@interact(
    dim1=widgets.Dropdown(options=dims, value=dims[0]),
    dim2=widgets.Dropdown(options=dims, value=dims[1]),
    index=widgets.IntSlider(min=0, max=(len(bunches[keys[0]]) - 1), value=0),
    bins=widgets.IntSlider(min=32, max=128, value=64),
)
def update(dim1: str, dim2: str, index: int, bins: int):
    if dim1 == dim2:
        return

    axis = [dims.index(dim) for dim in [dim1, dim2]]
    axis = tuple(axis)
    
    fig, axs = pplt.subplots(ncols=2, share=False, space=7.0)
    for ax, key in zip(axs, keys):
        X = bunches[key][index]
        ax.hist2d(
            X[:, axis[0]], 
            X[:, axis[1]], 
            bins=bins,
            range=[limits[axis[0]], limits[axis[1]]],
            cmap="viridis",
        )    
        paxs = [ax.panel_axes(loc) for loc in ["top", "right"]]
        for pax in paxs:
            pax.format(xspineloc="bottom", yspineloc="left")
        kws = dict(bins=bins, density=False, color="black", histtype="step", lw=1.25)
        paxs[0].hist( X[:, axis[0]], range=limits[axis[0]], **kws)
        paxs[1].histh(X[:, axis[1]], range=limits[axis[1]], **kws)
    
    axs.format(xlabel=labels[axis[0]], ylabel=labels[axis[1]])
    axs[0].format(title=f"turn = {bunch_turns[key][index]}")
    plt.show()